In [6]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# Load the CSV file
file_path = "combined_customers_balanced_large.csv"
data = pd.read_csv(file_path)

# prepare data
data = data[data['Day'] != 'Total']

X = data[['NumDeliveries', 'TotalDistance']].values 
Y = data['TotalDistance'].values

# Split into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
Y_val = torch.tensor(Y_val, dtype=torch.float32)

# Main Neural Network
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Random seeding
torch.manual_seed(49)
np.random.seed(49)

# Initialize model, loss, optimizer
input_size = X_train.shape[1]
hidden_size = 290  # Adjust as needed
model = NeuralNet(input_size, hidden_size)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(1518):
    # Forward propagation
    outputs = model(X_train)
    loss = criterion(outputs.squeeze(), Y_train)
    
    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Print progress
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/1051], Loss: {loss.item():.4f}')
print("Final Epoch Count:", epoch + 1)

# Evaluate on training data
with torch.no_grad():
    train_outputs = model(X_train)
    train_mse = mean_squared_error(train_outputs.squeeze().numpy(), Y_train.numpy())
print("Training MSE:", train_mse)

# Evaluate on validation data
with torch.no_grad():
    val_outputs = model(X_val)
    val_mse = mean_squared_error(val_outputs.squeeze().numpy(), Y_val.numpy())
print("Validation MSE:", val_mse)

Epoch [100/1051], Loss: 0.3887
Epoch [200/1051], Loss: 0.1056
Epoch [300/1051], Loss: 0.0372
Epoch [400/1051], Loss: 0.0154
Epoch [500/1051], Loss: 0.0075
Epoch [600/1051], Loss: 0.0041
Epoch [700/1051], Loss: 0.0024
Epoch [800/1051], Loss: 0.0014
Epoch [900/1051], Loss: 0.0009
Epoch [1000/1051], Loss: 0.0006
Epoch [1100/1051], Loss: 0.0004
Epoch [1200/1051], Loss: 0.0003
Epoch [1300/1051], Loss: 0.0002
Epoch [1400/1051], Loss: 0.0002
Epoch [1500/1051], Loss: 0.0002
Final Epoch Count: 1518
Training MSE: 0.0001503857
Validation MSE: 0.00015556271
